In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import CM4Xutils
CM4Xutils.__version__

'0.5.0'

In [3]:
zenodo_data_version = "0.0.7"
doi = "10.5281/zenodo.15420739"

zenodo_data_version = zenodo_data_version.replace(".", "_") # for cleaner filenames

In [4]:
import warnings
import dask
import xarray as xr
import xwmt
import xgcm
import matplotlib.pyplot as plt
import numpy as np
import cftime

import doralite
import gfdl_utils.core as gu

In [5]:
grid = CM4Xutils.load_wmt_grid(
    "CM4Xp25",
    interval=str(2000),
    dmget=True
)

Loading CM4Xp25-piControl for interval `2000`.
Issuing dmget command to migrate data to disk. Migration complete.


/work/hfd/.conda/envs/CM4X/lib/python3.12/site-packages/xgcm/grid_ufunc.py:836: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  out_dim: grid._ds.dims[out_dim] for arg in out_core_dims for out_dim in arg


Issuing dmget command to migrate data to disk. Migration complete.


/work/hfd/.conda/envs/CM4X/lib/python3.12/site-packages/xgcm/grid_ufunc.py:836: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  out_dim: grid._ds.dims[out_dim] for arg in out_core_dims for out_dim in arg
/work/hfd/.conda/envs/CM4X/lib/python3.12/site-packages/xgcm/grid_ufunc.py:836: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  out_dim: grid._ds.dims[out_dim] for arg in out_core_dims for out_dim in arg


Issuing dmget command to migrate data to disk. Migration complete.
Issuing dmget command to migrate data to disk. Migration complete.
Issuing dmget command to migrate data to disk. Migration complete.
Loading CM4Xp25-historical for interval `2000`.
Issuing dmget command to migrate data to disk. Migration complete.


/work/hfd/.conda/envs/CM4X/lib/python3.12/site-packages/xgcm/grid_ufunc.py:836: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  out_dim: grid._ds.dims[out_dim] for arg in out_core_dims for out_dim in arg


Issuing dmget command to migrate data to disk. Migration complete.


/work/hfd/.conda/envs/CM4X/lib/python3.12/site-packages/xgcm/grid_ufunc.py:836: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  out_dim: grid._ds.dims[out_dim] for arg in out_core_dims for out_dim in arg
/work/hfd/.conda/envs/CM4X/lib/python3.12/site-packages/xgcm/grid_ufunc.py:836: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  out_dim: grid._ds.dims[out_dim] for arg in out_core_dims for out_dim in arg


Issuing dmget command to migrate data to disk. Migration complete.
Issuing dmget command to migrate data to disk. Migration complete.
Issuing dmget command to migrate data to disk. Migration complete.
Overriding CM4Xp25 grid coordinates from supergrid.
Regridding ice
Inferring Z grid coordinate: depth `z_`


/work/hfd/.conda/envs/CM4X/lib/python3.12/site-packages/dask/core.py:127: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))


In [6]:
grid._ds.sigma2.attrs = {
    "long_name": "Potential Density referenced to 2000 dbar (minus 1000 kg/m3)",
    "units": "kg m-3",
    "cell_methods": "area:mean z_l:mean yh:mean xh:mean time:mean",
    "volume": "volcello",
    "area": "areacello",
    "time_avg_info": "average_T1,average_T2,average_DT",
    "description": "Computed offline using the gsw python package implementation of TEOS10.",
}

## Isolate minimal subset of surface variables for surface WMT calculations

In [7]:
ds_2d = grid._ds.copy().chunk(-1)
ds_2d["rsntds"] = ds_2d["rsdoabsorb"].sum("z_l")
two_dim_vars = [
    "tos", "sos", # state variables required for surface density
    "wfo", "prlq", "prsn", "evs", "friver", "ficeberg", "friver", "fsitherm", "vprec", # surface freshwater fluxes
    "hfds", "hflso", "hfsso", "rlntds", "rsntds", # surface heat fluxes
    "sfdsi",
]
ds_2d = ds_2d.drop_vars([v for v in ds_2d.data_vars if (v not in ds_2d.coords) and (v not in two_dim_vars)])
ds_2d = ds_2d.isel(z_l=0, drop=True).sel(time=ds_2d.time[0:12], exp="forced", drop=True)

In [8]:
ds_2d.attrs = {
    "description": "An example netcdf file containing the surface heat, freshwater, and salt fluxes required to compute surface-forced water mass transformations in MOM6. All variables are provided on the eddy-permitting native horizontal grid.",
    "version": zenodo_data_version,
    "doi": doi
}
ds_2d.to_netcdf(f"../data/MOM6_example_diags/MOM6_global_example_surface_fluxes_v{zenodo_data_version}.nc", mode="w")

## Compute the annual mean for the first year

In [9]:
drop_vars = ["average_DT", "average_T1", "average_T2", "vo", "uo"]
ds = grid._ds.sel(exp="forced", drop=True).isel(time=np.arange(0,12,1), time_bounds=np.arange(0,13,1)).drop_vars(drop_vars).fillna(0.)

In [10]:
def to_numeric_time(date):
    # Convert to number of days since 0001-01-01
    return cftime.date2num(date, units='days since 0001-01-01', calendar='noleap')

# Apply the function to your time array
numeric_time = np.vectorize(to_numeric_time)(ds.time_bounds.values)

# Convert days to seconds
seconds_array = numeric_time * 24 * 60 * 60

ds = ds.assign_coords({"time_delta":xr.DataArray(np.diff(seconds_array), dims=("time",))})

In [11]:
# Annual-mean, weighted by length of each month
# Note: da.weighted behaves strangely when there are NaNs. Fill the NaNs for correct behavior!
#ds_mean = ds.drop_dims("time_bounds").weighted(ds.time_delta).mean(dim="time", keep_attrs=True).chunk(-1)
ds_mean = ds.fillna(0.).drop_dims("time_bounds").weighted(ds.time_delta).mean(dim="time", keep_attrs=True).chunk(-1)
ds_mean = ds_mean.assign_coords({c:ds.coords[c] for c in ds.coords if c in ds_mean.coords})
ds_mean = ds_mean.expand_dims(["time"]).assign_coords({"time": xr.DataArray(ds.time_bounds[[6]].values, dims=("time",), attrs=ds.coords["time"].attrs)})
ds_mean = ds_mean.assign_coords({"geolon_c": ds.geolon_c, "geolat_c": ds.geolat_c})

In [12]:
exp = "historical"
t = "*2000*"
pp = doralite.dora_metadata(CM4Xutils.exp_dict["CM4Xp25"][exp])['pathPP']
ppname = gu.find_unique_variable(pp, "T_adx")
out = "ts"
local = gu.get_local(pp, ppname, out)
ds_mean_heat_transports = (
    gu.open_frompp(pp, ppname, out, local, t, ["T_adx", "T_ady"], chunks=-1, dmget=True)
    .drop_dims("nv").drop_vars(["average_DT", "average_T1", "average_T2"])
    .isel(time=[0])
)
ds_mean_heat_transports = ds_mean_heat_transports.assign_coords({c:ds_mean.coords[c] for c in ds_mean.coords})

ds_mean = xr.merge([ds_mean, ds_mean_heat_transports])

Issuing dmget command to migrate data to disk. Migration complete.


## Create the z-coordinate grid

In [13]:
coords={
    'X': {'center': 'xh',  'outer': 'xq'},
    'Y': {'center': 'yh',  'outer': 'yq'},
    'Z': {'center': 'z_l', 'outer': 'z_i'},
}
metrics = {
    ('X', 'Y'): ['areacello']
}
grid_mean = xgcm.Grid(
    ds_mean,
    coords=coords,
    metrics=metrics,
    boundary={"X":"periodic", "Y":"extend", "Z":"extend"},
    autoparse_metadata=False
)

### Save density, thickness, and mass transport variables in z-coordinates

In [14]:
ds_transports = ds_mean[["umo", "vmo"]]
ds_transports = ds_transports.assign_coords(ds_mean.coords)
ds_transports.attrs = {
    "description": "An example netcdf file containing annual-mean horizontal mass transports in a MOM6 simulation. All variables are provided on the eddy-permitting native horizontal grid but remapped online into depth coordinates from the hybrid vertical coordinate.",
    "version": zenodo_data_version,
    "doi": doi
}
ds_transports.to_netcdf(f"../data/MOM6_example_diags/MOM6_global_example_transport_zlevels_v{zenodo_data_version}.nc", mode="w")

In [15]:
ds_density = ds_mean[["sigma2", "thkcello"]]
ds_density = ds_density.assign_coords(ds_mean.coords)
ds_density.attrs = {
    "description": "An example netcdf file containing the annual-mean potential density field in a MOM6 simulation. All variables are provided on the eddy-permitting native horizontal grid but remapped online into depth coordinates from the hybrid vertical coordinate.",
    "version": zenodo_data_version,
    "doi": doi
}
ds_density.to_netcdf(f"../data/MOM6_example_diags/MOM6_global_example_sigma2_zlevels_v{zenodo_data_version}.nc", mode="w")

### Save vertically-integrated mass and heat budget diagnostics on the native horizontal grid

In [16]:
ds_mass = ds_mean[[
    "umo", "vmo", "dhdt", "dynamics_h_tendency", "vert_remap_h_tendency", "boundary_forcing_h_tendency",
]]
ds_mass = ds_mass.assign_coords(ds_mean.coords)
ds_mass = ds_mass.sum("z_l", keep_attrs=True).drop_dims(["z_i"]).compute()
ds_mass.attrs = {
    "description": "An example netcdf file containing all variables required to close the vertically-integrated mass budgets in MOM6. All variables are provided on the higher resolution (eddy-permitting) native grid.",
    "version": zenodo_data_version,
    "doi": doi
}
ds_mass.to_netcdf(f"../data/MOM6_example_diags/MOM6_global_example_vertically_integrated_mass_budget_v{zenodo_data_version}.nc", mode="w")

In [17]:
ds_heat = ds_mean[[
    "tos",
    "T_adx", "T_ady",
    "opottemptend", "T_advection_xy", "Th_tendency_vert_remap",
    "boundary_forcing_heat_tendency", "opottempdiff",
    "internal_heat_heat_tendency", "frazil_heat_tendency"
]]
ds_heat = ds_heat.assign_coords(ds_mean.coords)
ds_heat = ds_heat.sum("z_l", keep_attrs=True).drop_dims(["z_i"]).compute()
ds_heat.attrs = {
    "description": "An example netcdf file containing all variables required to close the vertically-integrated heat budgets in MOM6. All variables are provided on the higher resolution (eddy-permitting) native grid.",
    "version": zenodo_data_version,
    "doi": doi
}
ds_heat.to_netcdf(f"../data/MOM6_example_diags/MOM6_global_example_vertically_integrated_heat_budget_v{zenodo_data_version}.nc", mode="w")

### Save horizontally-coarsened diagnostics on all depth levels

In [18]:
ds_coarse = CM4Xutils.coarsen.horizontally_coarsen(ds_mean, grid_mean, {"X":6, "Y":6})
ds_coarse.attrs = {
    "description": "An example netcdf file containing all variables required to close mass, heat, salt, and thermodynamic density budgets in MOM6, as well as some other useful variables. All variables have been conservatively coarsened from the higher resolution (eddy-permitting) native grid.",
    "version": zenodo_data_version,
    "doi": doi
}
ds_coarse.to_netcdf(f"../data/MOM6_example_diags/MOM6_global_example_diagnostics_zlevels_v{zenodo_data_version}.nc", mode="w")

Skipping variable time_bnds because `cell_methods` attribute not defined.
